In [ ]:
from totalsegmentator.python_api import totalsegmentator

def main():
    input_path=r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\CT.nii.gz"
    output_path= r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\Total_Segmentator_Teeth"

    totalsegmentator(input_path,output_path,task = 'teeth')
    print('\nDone!')

if __name__ == "__main__":
 main()

In [9]:
#for combining teeth segmentations
import nibabel as nib
import numpy as np
from pathlib import Path

folder = Path(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\teeth_segmentations")

out_path = folder / "teeth.nii.gz"

nii_files = sorted(folder.glob("*.nii*"))

if not nii_files:
    raise ValueError("No NIfTI files found in the folder!")

ref_img = nib.load(nii_files[0])
combined_data = np.zeros(ref_img.shape, dtype=np.uint8)

for f in nii_files:
    img = nib.load(f)
    data = img.get_fdata()

    if data.shape != combined_data.shape:
        raise ValueError(f"Shape mismatch: {f.name} has shape {data.shape}, expected {combined_data.shape}")

    combined_data = np.logical_or(combined_data, data).astype(np.uint8)
    print(f"Added: {f.name}")

combined_img = nib.Nifti1Image(combined_data, affine=ref_img.affine, header=ref_img.header)
nib.save(combined_img, out_path)

print(f"\n Combined mask saved to: {out_path}")

Added: lower_left_canine_fdi33.nii.gz
Added: lower_left_canine_pulp_fdi133.nii.gz
Added: lower_left_central_incisor_fdi31.nii.gz
Added: lower_left_central_incisor_pulp_fdi131.nii.gz
Added: lower_left_first_molar_fdi36.nii.gz
Added: lower_left_first_molar_pulp_fdi136.nii.gz
Added: lower_left_first_premolar_fdi34.nii.gz
Added: lower_left_first_premolar_pulp_fdi134.nii.gz
Added: lower_left_lateral_incisor_fdi32.nii.gz
Added: lower_left_lateral_incisor_pulp_fdi132.nii.gz
Added: lower_left_second_molar_fdi37.nii.gz
Added: lower_left_second_molar_pulp_fdi137.nii.gz
Added: lower_left_second_premolar_fdi35.nii.gz
Added: lower_left_second_premolar_pulp_fdi135.nii.gz
Added: lower_left_third_molar_fdi38.nii.gz
Added: lower_left_third_molar_pulp_fdi138.nii.gz
Added: lower_right_canine_fdi43.nii.gz
Added: lower_right_canine_pulp_fdi143.nii.gz
Added: lower_right_central_incisor_fdi41.nii.gz
Added: lower_right_central_incisor_pulp_fdi141.nii.gz
Added: lower_right_first_molar_fdi46.nii.gz
Added: lower

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

ct_img = nib.load(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\CT.nii.gz")
X = ct_img.get_fdata()
X = (X - np.min(X)) / (np.max(X) - np.min(X))  # Normalize to [0, 1] (binary)

seg_paths = {
    #"mandible_manual": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\mandible_manual.nii.gz",
    #"mandible_totalseg": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\Total_Segmentator\mandible.nii.gz",
    #"Mandible_R": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\manual_segmentations\nii\Mandible_R.nii.gz",
    #"Mandible_R_(1)": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\manual_segmentations\nii\Mandible_R_(1).nii.gz",
    #"Dentition": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\manual_segmentations\nii\Dentition.nii.gz",
    #"teeth_upper": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\Total_Segmentator\teeth_upper.nii.gz",
    #"teeth_lower": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\Total_Segmentator\teeth_lower.nii.gz",
    "teeth": r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\teeth_segmentations\teeth.nii.gz"
}
segments = {name: nib.load(path).get_fdata() for name, path in seg_paths.items()}
spacing = ct_img.header.get_zooms()

overlay_colors = {
    #"mandible_manual": (0.2, 0.4, 1.0, 0.25),          # blueish
    #"mandible_totalseg": (1.0, 0.6, 0.0, 0.35),      # orange
    #"teeth": (0.2, 1.0, 0.2, 0.35), # green
    "teeth": (0.6, 0.2, 1.0, 0.35), # purple
    #"M1": (0.9, 0.9, 0.8, 0.25),         # bone
   # "teeth_upper": (1.0, 0.2, 0.2, 0.35),   # red
   # "teeth_lower": (1.0, 0.6, 0.4, 0.35),   # light red/orange
}

def show_slice(slice_index, plane, alpha=0.35, patient_id="UIDQQ0x1x1Hx7"):
    plt.figure(figsize=(6, 6))

    if plane == 'Axial':
        img_slice = np.rot90(X[:, :, slice_index])
        aspect = spacing[1] / spacing[0]
        seg_slices = {n: np.rot90(S[:, :, slice_index]) for n, S in segments.items()}

    elif plane == 'Coronal':
        img_slice = np.flipud(np.rot90(X[:, slice_index, :]))
        aspect = spacing[2] / spacing[0]
        seg_slices = {n: np.flipud(np.rot90(S[:, slice_index, :])) for n, S in segments.items()}

    elif plane == 'Sagittal':
        img_slice = np.flipud(np.rot90(X[slice_index, :, :]))
        aspect = spacing[2] / spacing[1]
        seg_slices = {n: np.flipud(np.rot90(S[slice_index, :, :])) for n, S in segments.items()}

    plt.imshow(img_slice, cmap='gray', origin='lower', aspect=aspect)

    for name, seg_slice in seg_slices.items():
        mask = seg_slice > 0.5
        if np.any(mask):
            rgba = list(overlay_colors[name])
            rgba[3] = alpha  
            overlay = np.zeros((*mask.shape, 4))
            overlay[mask] = rgba
            plt.imshow(overlay, origin='lower', aspect=aspect)

    plt.title(f"{plane} slice {slice_index}")
    plt.axis("off")


    legend_handles = [
        plt.Line2D([0], [0], color=overlay_colors[n][:3], lw=4, label=n)
        for n in overlay_colors
    ]
    leg=plt.legend(handles=legend_handles, loc='upper right', fontsize=8, frameon=False, labelcolor='white', title="Segments")
    leg.get_title().set_fontsize(10)
    leg.get_title().set_color('white')
    clear_output(wait=True)
    display(plt.gcf())
    plt.close()


plane_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='Plane:'
)

slice_slider = widgets.IntSlider(
    value=X.shape[2] // 2,
    min=0,
    max=X.shape[2] - 1,
    step=1,
    description='Slice:',
    continuous_update=True,
    layout=widgets.Layout(width='80%')
)

alpha_slider = widgets.FloatSlider(
    value=0.35,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Opacity:',
    continuous_update=True,
    layout=widgets.Layout(width='60%')
)


def update_slider_range(*args):
    plane = plane_dropdown.value
    if plane == 'Axial':
        slice_slider.max = X.shape[2] - 1
        slice_slider.value = X.shape[2] // 2
    elif plane == 'Coronal':
        slice_slider.max = X.shape[1] - 1
        slice_slider.value = X.shape[1] // 2
    elif plane == 'Sagittal':
        slice_slider.max = X.shape[0] - 1
        slice_slider.value = X.shape[0] // 2


plane_dropdown.observe(update_slider_range, names='value')
update_slider_range()

widgets.interact(show_slice, slice_index=slice_slider, plane=plane_dropdown, alpha=alpha_slider)


interactive(children=(IntSlider(value=122, description='Slice:', layout=Layout(width='80%'), max=244), Dropdow…

<function __main__.show_slice(slice_index, plane, alpha=0.35, patient_id='UIDQQ0x1x1Hx7')>